In [1]:
import getpass
import os

from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = getpass.getpass()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [20]:
# load document 
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
loader = WebBaseLoader(urls)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=500,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=0,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)
texts = text_splitter.split_documents(docs)

In [22]:
# vector db
from langchain_chroma import Chroma
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents=texts, embedding=embeddings)
retriever = db.as_retriever()
query = "agent memory"
retrieved_docs = retriever.invoke(query)

In [23]:
query

'agent memory'

In [24]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Determine whether query and doc has relevance. If query and doc has relevance, you say yes else no. \n{format_instructions}\n{query}\n{doc}",
    input_variables=["query", "doc"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": query, "doc": retrieved_docs[0].page_content})

{'relevance': 'yes'}

In [39]:
print(chain.invoke({"query": query, "doc": retrieved_docs[1].page_content}))
print(chain.invoke({"query": query, "doc": retrieved_docs[2].page_content}))
print(chain.invoke({"query": query, "doc": retrieved_docs[3].page_content}))

{'relevance': 'yes'}
{'relevance': 'yes'}
{'relevance': 'no'}


In [40]:
retrieved_docs[3].page_content

'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'

In [44]:
chain.invoke({"query": query, "doc": 'blah blah'})

{'relevance': 'no'}

In [51]:
# ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성 prompt | llm | parser 코드 작성
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model="gpt-4o-mini")
parser = JsonOutputParser()
prompt = PromptTemplate(
    template="Determine whether query and doc has relevance. If query and doc has relevance, you say yes else no\n{format_instructions}\n{query}\n{doc}",
    input_variables=["query", "doc"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"doc": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk)
    #print(chunk, end="", flush=True)

{}
{'relevance': ''}
{'relevance': 'yes'}


In [52]:
rag_chain.stream('test')

<generator object RunnableSequence.stream at 0x13fcdaac0>

In [49]:
type(retrieved_docs[0])

langchain_core.documents.base.Document